mnist_mlp.py

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

In [3]:
batch_size = 128
num_classes = 10
epochs = 20

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
469/469 [==============================] - 6s 11ms/step - loss: 0.4329 - accuracy: 0.8627 - val_loss: 0.1055 - val_accuracy: 0.9676
Epoch 2/20
469/469 [==============================] - 4s 9ms/step - loss: 0.1058 - accuracy: 0.9680 - val_loss: 0.0760 - val_accuracy: 0.9767
Epoch 3/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0727 - accuracy: 0.9781 - val_loss: 0.0750 - val_accuracy: 0.9769
Epoch 4/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0574 - accuracy: 0.9821 - val_loss: 0.0814 - val_accuracy: 0.9776
Epoch 5/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0488 - accuracy: 0.9846 - val_loss: 0.0814 - val_accuracy: 0.9796
Epoch 6/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0401 - accuracy: 0.9877 - val_loss: 0.0788 - val_accuracy: 0.9819
Epoch 7/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0345 - accuracy: 0.9894 - val_loss: 0.0848 - val_accuracy: 0.9801
Epoch

pi.py

In [9]:
import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

In [12]:
if __name__ == "__main__":
    """
        Usage: pi [partitions]
    """
    spark = SparkSession\
        .builder\
        .appName("PythonPi")\
        .getOrCreate()

    #partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
    partitions = 2
    n = 100000 * partitions

    def f(_):
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 <= 1 else 0

    count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    spark.stop()

Pi is roughly 3.152120
